In [1]:
import pandas as pd
# java = c_class
java = pd.read_csv('./c_class/c_class_mb.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
java_user_count = java.groupby('user_id').count().reset_index()[['user_id','item_id']]
java_inter = java[java['user_id'].isin(java_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()
java_inter['time_avg'] = java.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
java_inter['time_var'] = java.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
java_inter['time_max'] = java.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
java_inter['time_min'] = java.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
java_inter['inter_len'] = java_inter['item_id'].apply(lambda x : len(x))
java_inter = java_inter[java_inter['inter_len']>=3]
java_inter = java_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = java_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['scene'] = predict_y

beh_count = java.groupby(['user_id','behavior']).count().reset_index()
java_inter['easy_count'] = beh_count[beh_count['behavior']=='easy']['item_id']
java_inter['medium_count'] = beh_count[beh_count['behavior']=='medium']['item_id']
java_inter['hard_count'] = beh_count[beh_count['behavior']=='hard']['item_id']
java_inter['difficult_count'] = beh_count[beh_count['behavior']=='difficult']['item_id']
java_inter = java_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = java_inter[['easy_count','medium_count','hard_count','difficult_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
java_inter['cat'] = predict_y

alldata = []
for user in java_inter['user_id'].tolist():
    # print(user)
    seq_len = len(java_inter[java_inter['user_id']==user]['item_id'].item())
    total_seq = java_inter[java_inter['user_id']==user]['item_id'].item().tolist()
    scene = java_inter[java_inter['user_id']==user]['scene'].item()
    cat = java_inter[java_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
java_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
java_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,1,[57],3,0,60,[60]
1,5,[56],3,0,55,[55]
2,5,[56],3,0,57,"[55, 57]"
3,5,[56],3,0,58,"[55, 57, 58]"
4,5,"[56, 55]",3,0,57,[57]
...,...,...,...,...,...,...
28944,4405,[55],0,0,56,[56]
28945,4405,[55],0,0,57,"[56, 57]"
28946,4405,"[55, 56]",0,0,57,[57]
28947,4440,[55],0,0,57,[57]


In [2]:
java_new[java_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[57],3,0,60,[60]
1,5,[56],3,0,55,[55]
2,5,[56],3,0,57,"[55, 57]"
3,5,[56],3,0,58,"[55, 57, 58]"
4,5,"[56, 55]",3,0,57,[57]
...,...,...,...,...,...,...
28944,4405,[55],0,0,56,[56]
28945,4405,[55],0,0,57,"[56, 57]"
28946,4405,"[55, 56]",0,0,57,[57]
28947,4440,[55],0,0,57,[57]


In [3]:
java_test = java_new[java_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.1)
java_train = java_new[~java_new.index.isin(java_test)]
java_test

,user_id,inter_seq,scene,cat,target,pred_path
25069,2446,[55],0,0,56,[56]
17556,1930,"[55, 56, 57, 58]",2,0,62,"[59, 60, 61, 62]"
269,52,"[55, 56, 57, 58, 59, 61]",1,0,60,"[62, 60]"
26355,2555,"[55, 56, 57, 58, 59]",2,2,61,"[60, 61]"
3175,190,"[55, 56, 57, 58, 59]",1,1,60,[60]
...,...,...,...,...,...,...
21642,2196,"[55, 56, 57]",2,3,58,[58]
4377,250,"[55, 56]",3,1,57,[57]
7245,433,"[55, 56, 57, 58, 59]",1,3,61,"[60, 61]"
23753,2340,[61],2,1,60,"[55, 57, 56, 58, 60]"


In [4]:
java_test.to_csv('c_class/c_class_test_10.csv',header=True,sep='\t',index=False)
java_train.to_csv('c_class/c_class_train_10.csv',header=True,sep='\t',index=False)

In [5]:
import pandas as pd
# net = computer
net = pd.read_csv('./computer/computer_mb.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
net_user_count = net.groupby('user_id').count().reset_index()[['user_id','item_id']]
net_inter = net[net['user_id'].isin(net_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

net_inter['time_avg'] = net.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
net_inter['time_var'] = net.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
net_inter['time_max'] = net.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
net_inter['time_min'] = net.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
net_inter['inter_len'] = net_inter['item_id'].apply(lambda x : len(x))
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = net_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['scene'] = predict_y

beh_count = net.groupby(['user_id','behavior']).count().reset_index()
net_inter['easy_count'] = beh_count[beh_count['behavior']=='easy']['item_id']
net_inter['medium_count'] = beh_count[beh_count['behavior']=='medium']['item_id']
net_inter['hard_count'] = beh_count[beh_count['behavior']=='hard']['item_id']
net_inter['difficult_count'] = beh_count[beh_count['behavior']=='difficult']['item_id']
net_inter = net_inter[net_inter['inter_len']>=3]
net_inter = net_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = net_inter[['easy_count','medium_count','hard_count','difficult_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
net_inter['cat'] = predict_y

alldata = []
for user in net_inter['user_id'].tolist():
    # print(user)
    seq_len = len(net_inter[net_inter['user_id']==user]['item_id'].item())
    total_seq = net_inter[net_inter['user_id']==user]['item_id'].item().tolist()
    scene = net_inter[net_inter['user_id']==user]['scene'].item()
    cat = net_inter[net_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
net_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
net_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,1,[122],2,2,123,[123]
1,1,[122],2,2,128,"[123, 128]"
2,1,[122],2,2,129,"[123, 128, 129]"
3,1,[122],2,2,134,"[123, 128, 129, 134]"
4,1,[122],2,2,140,"[123, 128, 129, 134, 140]"
...,...,...,...,...,...,...
259707,8282,[126],0,2,159,"[131, 159]"
259708,8282,[126],0,2,135,"[131, 159, 135]"
259709,8282,"[126, 131]",0,2,159,[159]
259710,8282,"[126, 131]",0,2,135,"[159, 135]"


In [6]:
net_new[net_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,1,[122],2,2,123,[123]
1,1,[122],2,2,128,"[123, 128]"
2,1,[122],2,2,129,"[123, 128, 129]"
3,1,[122],2,2,134,"[123, 128, 129, 134]"
4,1,[122],2,2,140,"[123, 128, 129, 134, 140]"
...,...,...,...,...,...,...
259707,8282,[126],0,2,159,"[131, 159]"
259708,8282,[126],0,2,135,"[131, 159, 135]"
259709,8282,"[126, 131]",0,2,159,[159]
259710,8282,"[126, 131]",0,2,135,"[159, 135]"


In [7]:
net_test = net_new[net_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.12)
net_train = net_new[~net_new.index.isin(net_test)]
net_test

,user_id,inter_seq,scene,cat,target,pred_path
47737,5036,"[122, 134, 137, 139, 140, 123, 157, 142, 144, ...",2,0,153,[153]
182701,6702,"[122, 134, 123, 142, 144, 140, 139, 137, 151]",2,1,152,"[138, 128, 163, 148, 149, 152]"
84965,5422,"[122, 140, 139]",2,0,137,"[134, 123, 137]"
194299,6898,"[122, 134, 123, 137, 140]",2,3,139,[139]
201813,6978,"[122, 134, 123]",2,2,128,"[137, 139, 140, 142, 144, 128]"
...,...,...,...,...,...,...
83659,5412,"[122, 134]",2,2,123,"[137, 139, 123]"
223362,7200,"[122, 139, 123]",2,2,144,"[142, 134, 137, 140, 144]"
71277,5340,"[122, 123, 137, 134, 142, 128, 151, 139, 140, ...",2,1,150,"[158, 150]"
150021,5921,[122],2,2,149,"[123, 137, 141, 142, 128, 151, 163, 149]"


In [8]:
net_test.to_csv('computer/computer_test_10.csv',header=True,sep='\t',index=False)
net_train.to_csv('computer/computer_train_10.csv',header=True,sep='\t',index=False)

In [10]:
import pandas as pd
# os = data_science
os = pd.read_csv('./data_science/data_sci_mb.txt', sep='\t', names=['user_id','item_id','behavior','timestamp'])
os_user_count = os.groupby('user_id').count().reset_index()[['user_id','item_id']]
os_inter = os[os['user_id'].isin(os_user_count['user_id'])].groupby('user_id').agg('item_id').unique().reset_index()

os_inter['time_avg'] = os.groupby('user_id')['timestamp'].mean().reset_index()['timestamp']
os_inter['time_var'] = os.groupby('user_id')['timestamp'].var().reset_index()['timestamp']
os_inter['time_max'] = os.groupby('user_id')['timestamp'].max().reset_index()['timestamp']
os_inter['time_min'] = os.groupby('user_id')['timestamp'].min().reset_index()['timestamp']
os_inter['inter_len'] = os_inter['item_id'].apply(lambda x : len(x))
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)

from sklearn import preprocessing
from sklearn.cluster import KMeans

k_model = KMeans(n_clusters=4)
train_x = os_inter[['time_avg','time_var','time_max','time_min','inter_len']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['scene'] = predict_y

beh_count = os.groupby(['user_id','behavior']).count().reset_index()
os_inter['easy_count'] = beh_count[beh_count['behavior']=='easy']['item_id']
os_inter['medium_count'] = beh_count[beh_count['behavior']=='medium']['item_id']
os_inter['hard_count'] = beh_count[beh_count['behavior']=='hard']['item_id']
os_inter['difficult_count'] = beh_count[beh_count['behavior']=='difficult']['item_id']
os_inter = os_inter[os_inter['inter_len']>=3]
os_inter = os_inter.fillna(0)

k_model = KMeans(n_clusters=5)
train_x = os_inter[['easy_count','medium_count','hard_count','difficult_count']]
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
k_model.fit(train_x)
predict_y = k_model.predict(train_x)
os_inter['cat'] = predict_y

alldata = []
for user in os_inter['user_id'].tolist():
    # print(user)
    seq_len = len(os_inter[os_inter['user_id']==user]['item_id'].item())
    total_seq = os_inter[os_inter['user_id']==user]['item_id'].item().tolist()
    scene = os_inter[os_inter['user_id']==user]['scene'].item()
    cat = os_inter[os_inter['user_id']==user]['cat'].item()
    for i in range(1,seq_len):
        for j in range(1,seq_len-i):
            data = []
            inter_seq = total_seq[:i]
            pred_path = total_seq[i:i+j]
            if pred_path == []:
                continue
            target = pred_path[-1]
            data.append(user)
            data.append(inter_seq)
            data.append(scene)
            data.append(cat)
            data.append(target)
            data.append(pred_path)
            alldata.append(data)
os_new = pd.DataFrame(alldata,columns = ['user_id','inter_seq','scene','cat','target','pred_path'])
os_new

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,user_id,inter_seq,scene,cat,target,pred_path
0,2,[128],1,1,136,[136]
1,2,[128],1,1,125,"[136, 125]"
2,2,[128],1,1,123,"[136, 125, 123]"
3,2,[128],1,1,135,"[136, 125, 123, 135]"
4,2,[128],1,1,127,"[136, 125, 123, 135, 127]"
...,...,...,...,...,...,...
121106,8740,"[136, 123, 124]",0,2,125,"[126, 125]"
121107,8740,"[136, 123, 124]",0,2,127,"[126, 125, 127]"
121108,8740,"[136, 123, 124, 126]",0,2,125,[125]
121109,8740,"[136, 123, 124, 126]",0,2,127,"[125, 127]"


In [11]:
os_new[os_new['pred_path'].apply(lambda x: len(x))<=10]

,user_id,inter_seq,scene,cat,target,pred_path
0,2,[128],1,1,136,[136]
1,2,[128],1,1,125,"[136, 125]"
2,2,[128],1,1,123,"[136, 125, 123]"
3,2,[128],1,1,135,"[136, 125, 123, 135]"
4,2,[128],1,1,127,"[136, 125, 123, 135, 127]"
...,...,...,...,...,...,...
121106,8740,"[136, 123, 124]",0,2,125,"[126, 125]"
121107,8740,"[136, 123, 124]",0,2,127,"[126, 125, 127]"
121108,8740,"[136, 123, 124, 126]",0,2,125,[125]
121109,8740,"[136, 123, 124, 126]",0,2,127,"[125, 127]"


In [12]:
os_test = os_new[os_new['pred_path'].apply(lambda x: len(x))<=10].sample(frac=0.1)
os_train = os_new[~os_new.index.isin(os_test)]
os_test

,user_id,inter_seq,scene,cat,target,pred_path
56622,4461,"[136, 123]",3,1,124,[124]
70140,4914,"[135, 127, 124]",2,1,129,"[126, 132, 133, 134, 129]"
49197,3871,[136],3,3,134,"[125, 126, 123, 124, 129, 134]"
74219,5085,"[123, 124, 126, 128, 135, 127]",2,0,131,"[129, 131]"
100591,6580,"[123, 124, 126, 125, 127]",2,2,128,"[135, 136, 128]"
...,...,...,...,...,...,...
43362,3027,[136],3,2,128,"[123, 125, 126, 124, 131, 129, 128]"
42497,2920,"[136, 128, 123]",3,2,124,"[125, 126, 124]"
70800,4928,"[127, 135, 124, 126, 132, 133]",2,1,130,"[134, 130]"
81347,5340,"[123, 125, 127]",2,1,128,"[135, 128]"


In [13]:
os_test.to_csv('data_science/data_sci_test_10.csv',header=True,sep='\t',index=False)
os_train.to_csv('data_science/data_sci_train_10.csv',header=True,sep='\t',index=False)